In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 1. 기존 리포지토리 폴더로 이동
import os
os.chdir('/content/drive/MyDrive/Codeit_AI_4th_Drug_image_CV_project')

In [ ]:
# 2. 경로 확인
!pwd

['/content/drive/MyDrive/Codeit_AI_4th_Drug_image_CV_project']

작업 시 !git stash부터 !git status 제외하고 내용 수행 함

In [ ]:
# 3. (pull이 그냥 안될때 사용)현재 변경 사항 스태시
!git stash

In [ ]:
# 3. 최신 변경사항 가져오기
!git pull origin main

In [ ]:
# 4. 현재 상태 확인
!git status

In [ ]:
import os

# 경로 설정
project_path = "/content/drive/MyDrive/Codeit_AI_4th_Drug_image_CV_project"
train_img_path = f"{project_path}/data/train_images"
train_ann_path = f"{project_path}/data/train_annotations"
test_img_path = f"{project_path}/data/test_images"
yolo_path = f"{project_path}/data/yolo_dataset_final"

os.makedirs(f"{yolo_path}/images/train", exist_ok=True)
os.makedirs(f"{yolo_path}/images/val", exist_ok=True)
os.makedirs(f"{yolo_path}/labels/train", exist_ok=True)
os.makedirs(f"{yolo_path}/labels/val", exist_ok=True)

### JSON 병합 : 이미지 기준으로 annotation 병합

In [ ]:
import json
from pathlib import Path
from glob import glob
from collections import defaultdict
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

def collect_json_files(json_path=train_ann_path):
    """
    주어진 annotation 경로에서 모든 COCO 형식 JSON 파일 경로를 수집합니다.

    이 함수는 다음과 같은 구조를 가진 디렉토리에서:
        json_path/
            └── K-123456-789012-345678-901234_json/
                ├── K-123456/
                │   └── *.json
                ├── K-789012/
                │   └── *.json
                ...

    내부에 존재하는 모든 JSON 파일을 재귀적으로 탐색하여 리스트로 반환합니다.

    Args:
        json_path (str or Path, optional): annotation 디렉토리 경로. 기본값은 train_ann_path.

    Returns:
        list[Path]: 수집된 모든 JSON 파일 경로의 리스트.
    """
    json_files = []
    for folder in Path(json_path).glob("K-*"):
        for code_folder in folder.glob("*"):
            json_files.extend(code_folder.glob("*.json"))
    return json_files

def process_json_file(json_path):
    """
    단일 COCO 형식의 JSON 파일을 처리하여 이미지 정보, 어노테이션, 카테고리 정보를 추출합니다.

    이 함수는 주어진 JSON 파일을 열어 다음의 정보를 파싱합니다:
        - 이미지 파일 이름
        - 이미지 크기 (width, height)
        - 각 객체에 대한 bounding box (YOLO 형식으로 변환)
        - category_id → category_name 매핑 정보

    Args:
        json_path (str or Path): 처리할 COCO 포맷 JSON 파일 경로

    Returns:
        tuple:
            - img_name (str): 이미지 파일 이름
            - (image_info, anns) (tuple): 이미지 정보 딕셔너리와 어노테이션 리스트
            - local_category_map (dict): {category_id: category_name} 형태의 딕셔너리

    Raises:
        예외 발생 시 오류 메시지를 출력하고 None을 반환합니다.
    """
    try:
        with open(json_path, "r", encoding="utf-8") as f:
            data = json.load(f)

        image_info = data["images"][0]
        anns = data["annotations"]
        cats = data["categories"]

        img_name = image_info["file_name"]
        width = image_info["width"]
        height = image_info["height"]

        txt_lines = []

        for ann in anns:
            coco_cat_id = ann["category_id"]
            x, y, w, h = ann["bbox"]
            x_center = (x + w / 2) / width
            y_center = (y + h / 2) / height
            w /= width
            h /= height
            txt_lines.append((coco_cat_id, f"{x_center} {y_center} {w} {h}"))

        # ✅ 어노테이션 정보를 딕셔너리로 수집
        local_category_map = {cat["id"]: cat["name"] for cat in cats}

        return img_name, (image_info, anns), local_category_map

    except Exception as e:
        print(f"Error processing {json_path}: {e}")
        return None

def merge_annotations_by_image(json_path=train_ann_path, max_workers=8):
    """
    여러 COCO JSON 파일에서 동일한 이미지에 대한 어노테이션을 병합하고,
    전체 카테고리 정보를 통합하여 반환합니다.

    이 함수는 다음 작업을 수행합니다:
        - 주어진 annotation 디렉토리 내 모든 JSON 파일을 수집합니다.
        - 병렬(ThreadPoolExecutor)로 각 JSON 파일을 처리하여 이미지 이름별로 어노테이션을 그룹화합니다.
        - 이미지 파일명을 기준으로 여러 JSON 파일에 흩어져 있는 어노테이션을 하나로 병합합니다.
        - 전체 category_id와 category_name의 매핑 정보를 통합합니다.

    Args:
        json_path (str or Path, optional): COCO 형식의 어노테이션 JSON 파일들이 있는 최상위 디렉토리 경로. 기본값은 train_ann_path.
        max_workers (int): 병렬 처리를 위한 최대 스레드 수. 기본값은 8.

    Returns:
        tuple:
            - image_to_annots (dict): {이미지 파일명: [(image_info, annotations), ...]} 형태의 딕셔너리
            - all_categories (dict): {category_id: category_name} 형태의 전체 카테고리 매핑 정보

    Raises:
        예외 발생 시 오류 메시지와 traceback을 출력합니다.
    """
    try:
        json_files = collect_json_files(json_path)
        print(f"총 JSON 파일 수: {len(json_files)}")

        image_to_annots = defaultdict(list)
        all_categories = {}

        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            futures = {executor.submit(process_json_file, path): path for path in json_files}

            for future in tqdm(as_completed(futures), total=len(futures), desc="병합 중"):
                result = future.result()
                if result is None:
                    continue

                img_name, image_and_anns, local_categories = result

                # 어노테이션 병합
                image_to_annots[img_name].append(image_and_anns)

                # 카테고리 병합
                for cat_id, cat_name in local_categories.items():
                    all_categories[cat_id] = cat_name

        return image_to_annots, all_categories

    except Exception as e:
        print(f"❌ 병합 중 오류 발생: {e}")
    import traceback
    traceback.print_exc()

### COCO → YOLO 변환 및 파일 복사

In [ ]:
import os
import cv2
import shutil
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

def coco_to_yolo_bbox(bbox, img_w, img_h):
    """
    COCO 형식의 바운딩 박스를 YOLO 형식으로 변환합니다.

    Args:
        bbox (list or tuple): COCO 형식의 바운딩 박스 [x_min, y_min, width, height].
                              (픽셀 단위)
        img_w (int or float): 이미지의 너비 (픽셀 단위).
        img_h (int or float): 이미지의 높이 (픽셀 단위).

    Returns:
        list: YOLO 형식의 바운딩 박스 [x_center, y_center, width, height],
              모든 값은 이미지 크기로 정규화된 0~1 사이의 값입니다.

    Notes:
        - COCO: (x_min, y_min, width, height)
        - YOLO: (x_center, y_center, width, height), 모두 정규화된 비율 값
    """
    x, y, w, h = bbox
    x_center = (x + w / 2) / img_w
    y_center = (y + h / 2) / img_h
    return [x_center, y_center, w / img_w, h / img_h]

def process_image(key, entries, category_id_map, output_image_path, output_label_path, image_root):
    """
    이미지 파일 하나와 해당 이미지에 대한 모든 COCO 어노테이션을 YOLO 포맷으로 변환하고,
    해당 이미지를 지정된 디렉토리로 복사한 후 라벨 텍스트 파일을 생성합니다.

    이 함수는 병합된 어노테이션 정보(`entries`)를 기반으로:
        - 해당 이미지 파일을 원본 디렉토리에서 읽어옴
        - 모든 어노테이션을 YOLO 형식(x_center, y_center, w, h)으로 변환
        - YOLO 형식 라벨을 텍스트 파일로 저장
        - 원본 이미지를 지정된 출력 디렉토리로 복사

    Args:
        key (str): 이미지 파일명에서 확장자를 제외한 base 이름 (e.g., "K-123456-...")
        entries (list): [(image_info, annotations), ...] 형태의 어노테이션 리스트
        category_id_map (dict): COCO의 category_id → YOLO 인덱스(0부터 시작) 매핑 딕셔너리
        output_image_path (str): YOLO 학습용 이미지가 저장될 경로
        output_label_path (str): YOLO 라벨(.txt) 파일이 저장될 경로
        image_root (str): 원본 이미지가 있는 디렉토리 경로

    Returns:
        None. 해당 이미지와 라벨 파일이 지정된 위치에 저장됨.

    Raises:
        이미지가 없거나, 읽기 실패 시 경고 메시지를 출력하고 건너뜀.
    """
    # entries 중 하나라도 image_info["file_name"]을 통해 확장자 얻기
    # 예: entries = [(image_info, annotations), ...]
    # 여기서 image_info["file_name"] 꺼내기
    first_image_info = entries[0][0]
    file_name = first_image_info["file_name"]
    base, ext = os.path.splitext(file_name)
    # key가 base와 같다고 가정하면 ext 가지고 이미지 가져오기
    img_path = os.path.join(image_root, file_name)  # 더 확실한 방법

    if not os.path.exists(img_path):
        print(f"❌ 이미지 없음: {img_path}")
        return

    img = cv2.imread(img_path)
    if img is None:
        print(f"❌ 이미지 읽기 실패: {img_path}")
        return
    img_h, img_w = img.shape[:2]

    yolo_lines = []
    for image_data, annotations in entries:
        for ann in annotations:
            bbox = ann['bbox']
            category_id = ann['category_id']
            yolo_cls = category_id_map[category_id]
            yolo_bbox = coco_to_yolo_bbox(bbox, img_w, img_h)
            yolo_lines.append(f"{yolo_cls} {' '.join(map(str, yolo_bbox))}")

    os.makedirs(output_image_path, exist_ok=True)
    os.makedirs(output_label_path, exist_ok=True)
    dest_img_path = os.path.join(output_image_path, file_name)
    # copy
    shutil.copy(img_path, dest_img_path)

    label_path = os.path.join(output_label_path, base + ".txt")
    with open(label_path, 'w') as f:
        f.write("\n".join(yolo_lines))

def convert_to_yolo_format(image_to_annots, all_categories, category_id_map, output_image_path, output_label_path, image_root):
    """
    COCO 형식 어노테이션 데이터를 YOLO 학습 형식으로 변환하여 이미지와 라벨을 저장합니다.
    각 이미지를 병렬로 처리하여 변환 속도를 향상시킵니다.

    Args:
        image_to_annots (dict): 이미지 이름(base_key)을 키로 하고,
                                (image_info, annotations) 튜플들의 리스트를 값으로 가지는 딕셔너리.
                                merge_annotations_by_image() 함수의 출력 중 하나입니다.
        all_categories (dict): COCO category_id → category_name 매핑 딕셔너리 (사용되지 않지만 유지됨).
        category_id_map (dict): COCO category_id → YOLO class ID로 변환하는 매핑 딕셔너리.
        output_image_path (str): YOLO 학습용 이미지가 저장될 디렉토리 경로.
        output_label_path (str): YOLO 학습용 라벨(.txt)이 저장될 디렉토리 경로.
        image_root (str): 원본 이미지들이 저장된 상위 디렉토리 경로.

    Returns:
        None

    Notes:
        - process_image() 함수를 스레드풀로 병렬 실행하여 처리 속도를 향상시킵니다.
        - 이미지 단위로 예외 처리하여, 일부 실패해도 전체 작업은 계속됩니다.
        - YOLO 라벨은 [image_name].txt 형식으로 저장되며, 이미지도 함께 복사됩니다.
    """
    tasks = []
    with ThreadPoolExecutor() as executor:
        futures = {
            executor.submit(
                process_image, key, entries,
                category_id_map, output_image_path,
                output_label_path, image_root
            ): key
            for key, entries in image_to_annots.items()
        }

        for future in tqdm(as_completed(futures), total=len(futures), desc="Converting"):
            try:
                future.result()  # 예외 발생 시 잡기
            except Exception as e:
                key = futures[future]
                print(f"Error processing {key}: {e}")

### 이미지 학습/검증용 분할하는 키 생성

In [ ]:
import random

def split_dataset(image_to_annots, train_ratio=0.8):
    """
    이미지 키 목록을 학습용(train)과 검증용(val)으로 랜덤하게 분할합니다.

    Args:
        image_to_annots (dict): 이미지 파일명(확장자 없는 base key)을 키로 하는 어노테이션 딕셔너리.
                                보통 merge_annotations_by_image() 함수의 결과입니다.
        train_ratio (float): 학습 데이터셋에 사용할 비율 (기본값: 0.8).
                             0~1 사이의 값으로, 나머지는 검증 데이터로 사용됩니다.

    Returns:
        tuple:
            - train_keys (list): 학습용 데이터로 사용할 이미지 key 리스트.
            - val_keys (list): 검증용 데이터로 사용할 이미지 key 리스트.

    Notes:
        - 입력 딕셔너리의 키 목록을 무작위로 셔플한 후 비율에 따라 나눕니다.
        - 키는 보통 이미지 파일명에서 확장자를 제거한 문자열입니다.
    """
    keys = list(image_to_annots.keys())
    random.shuffle(keys)
    split_idx = int(len(keys) * train_ratio)
    return keys[:split_idx], keys[split_idx:]

### dataset.yaml 생성기 (YOLO 인덱스 매핑 포함)

In [ ]:
def save_dataset_yaml(yolo_path, categories_map):
    """
    YOLO 학습을 위한 dataset.yaml 파일을 생성하여 저장합니다.

    Args:
        yolo_path (str): YOLO 데이터셋이 저장된 최상위 디렉토리 경로.
                         이 경로 아래에 'images/train', 'images/val' 구조가 있어야 합니다.
        categories_map (dict): category_id → category_name 매핑 딕셔너리.
                               예: {0: 'pill_a', 1: 'pill_b', ...}

    Returns:
        None

    Notes:
        - 생성되는 dataset.yaml은 YOLO 학습 시 사용하는 config 파일입니다.
        - YAML에는 train/val 이미지 경로, 클래스 수(nc), 클래스 이름 목록(names)이 포함됩니다.
        - 클래스 이름은 category_id 순서대로 정렬되어 저장됩니다.
        - 출력 경로는: {yolo_path}/dataset.yaml
    """
    id_name_pairs = sorted(categories_map.items(), key=lambda x: x[0])
    names = [name for _, name in id_name_pairs]
    yaml_content = f"""
                    train: {yolo_path}/images/train
                    val: {yolo_path}/images/val
                    nc: {len(names)}
                    names: {names}
                    """
    with open(f"{yolo_path}/dataset.yaml", "w") as f:
        f.write(yaml_content)

### YOLOv8 학습

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.0 MB/s eta 0:00:00


In [ ]:
import torch

# 학습 설정
training_config = {
    'data': f"{yolo_path}/dataset.yaml",           # 데이터셋 설정 파일
    'epochs': 5,                # 에포크 수 (테스트용으로 작게 설정)
    'imgsz': 1280,              # 이미지 크기 (파이프라인과 일치)
    'batch': 8,                 # 배치 크기 (GPU 메모리에 따라 조정)
    'device': 0 if torch.cuda.is_available() else 'cpu',  # GPU 사용
    'project': 'result/train',    # 결과 저장 폴더
    'name': 'pill_detection',   # 실험 이름
    'save': True,               # 모델 저장
    'plots': True,              # 학습 그래프 생성
    'verbose': True,             # 상세 로그 출력
    # 추가: 데이터 증강
    'degrees': 10,
    'scale': 0.5,
    'shear': 2.0,
    'hsv_h': 0.015,
    'hsv_s': 0.7,
    'hsv_v': 0.4,
    'augment': True,
}

print("학습 설정:")
for key, value in training_config.items():
    print(f"  {key}: {value}")

학습 설정:
  data: /content/drive/MyDrive/Codeit_AI_4th_Drug_image_CV_project/data/yolo_dataset_final/dataset.yaml
  epochs: 5
  imgsz: 1280
  batch: 8
  device: 0
  project: result/train
  name: pill_detection
  save: True
  plots: True
  verbose: True
  degrees: 10
  scale: 0.5
  shear: 2.0
  hsv_h: 0.015
  hsv_s: 0.7
  hsv_v: 0.4
  augment: True


In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8m.pt") # YOLOv8m 모델 사용

In [ ]:
try:
    train_results = model.train(**training_config)
    print("✅ 학습 완료!")

    # 학습 결과 확인 (속성 탐색)
    print(f"\n학습 결과 정보:")
    print(f"  결과 타입: {type(train_results)}")

    # 사용 가능한 속성들 확인
    if hasattr(train_results, '__dict__'):
        attrs = [attr for attr in dir(train_results) if not attr.startswith('_')]
        print(f"  사용 가능한 속성: {attrs[:5]}...")  # 처음 5개만 표시

    # 일반적인 YOLO 결과 접근 방법
    try:
        # 최종 메트릭 정보는 보통 마지막 에포크 결과에서 확인
        print(f"\n학습 요약:")
        print(f"  총 에포크: {training_config['epochs']}")
        print(f"  사용된 배치 크기: {training_config['batch']}")
        print(f"  이미지 크기: {training_config['imgsz']}")

        # 학습이 완료되었다는 것 자체가 성공
        print(f"  상태: 학습 완료")

    except Exception as e:
        print(f"결과 정보 추출 중 오류: {e}")

except Exception as e:
    print(f"❌ 학습 중 오류 발생: {e}")
    import traceback
    traceback.print_exc()

Ultralytics 8.3.203 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/Codeit_AI_4th_Drug_image_CV_project/data/yolo_dataset_final/dataset.yaml, degrees=10, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1280, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=pill_detection14, nbs=64, nms=False, opset=None, optimize=False, optimizer=au

#### (추가 내용) 학습 부분을 train.py로 바꾼다면 실행하는 방식
실행하지 않은 내용, 혹시 정리에 도움이 될까 해서 작성

In [ ]:
# train.py 내용 - 실행하지 않음
import os
import torch
from ultralytics import YOLO

# 경로 설정
yolo_path = os.path.dirname(os.path.abspath(__file__))  # 현재 경로 기준

# 학습 설정
training_config = {
    'data': f"{yolo_path}/dataset.yaml",  # 데이터셋 YAML
    'epochs': 100,
    'imgsz': 1280,
    'batch': 8,
    'device': 0 if torch.cuda.is_available() else 'cpu',
    'project': 'result/train',
    'name': 'pill_detection',
    'save': True,
    'plots': True,
    'verbose': True
}

def main():
    print("📌 YOLOv8 학습 시작")

    # 설정 출력
    print("🛠 학습 설정:")
    for k, v in training_config.items():
        print(f"  {k}: {v}")

    # 모델 로드
    print("\n📥 모델 로드 중...")
    model = YOLO("yolov8m.pt")  # ultralytics에서 자동 다운로드됨

    try:
        # 학습 수행
        print("\n🚀 학습 실행 중...")
        train_results = model.train(**training_config)

        # 학습 결과 요약
        print("\n✅ 학습 완료!")
        print(f"  저장 경로: {model.train_args.project}/{model.train_args.name}")
        print(f"  최종 모델: {model.train_args.project}/{model.train_args.name}/weights/best.pt")

    except Exception as e:
        print(f"❌ 학습 중 오류 발생: {e}")
        import traceback
        traceback.print_exc()

if __name__ == '__main__':
    main()

실행 방법 (터미널/CLI)  
python train.py

실행 후 결과가 result/train/pill_detection/weights/best.pt에 자동 저장

#### (추가 내용) 실험 결과 확인 부분을 함수 형태로 정의하는 부분과 실행하는 부분으로 분리

In [ ]:
import os
import glob
import pandas as pd

def analyze_latest_yolo_training_result(base_dir='result/train'):
    """
    지정된 디렉토리 내에서 가장 최근의 YOLO 학습 결과를 찾아
    학습 모델(best.pt, last.pt) 경로와 주요 성능 지표를 분석 및 출력합니다.

    이 함수는 다음과 같은 작업을 수행합니다:
        - base_dir 하위에서 가장 최근 생성된 'pill_detection*' 폴더를 찾습니다.
        - 해당 폴더 내 'weights' 디렉토리에서 best.pt 및 last.pt 경로를 확인합니다.
        - results.csv 파일에서 마지막 에포크의 주요 학습/검증 메트릭을 추출하여 요약 출력합니다.

    Args:
        base_dir (str): YOLO 학습 결과가 저장된 기본 디렉토리 경로 (기본값: 'result/train').

    Returns:
        dict: {
            'latest_folder': str,            # 가장 최근 학습 결과 폴더 경로
            'best_model_path': str or None,  # best.pt 경로 (존재할 경우)
            'last_model_path': str or None,  # last.pt 경로 (존재할 경우)
            'metrics': dict                  # 주요 성능 지표 요약 (예: mAP50, box loss 등)
        }

    Raises:
        - 학습 결과가 존재하지 않으면 경고 메시지를 출력하고 빈 결과를 반환합니다.
        - results.csv를 읽는 중 오류가 발생하면 경고 메시지를 출력하고 무시합니다.
    """
    result = {
        'latest_folder': None,
        'best_model_path': None,
        'last_model_path': None,
        'metrics': {}
    }

    train_folders = glob.glob(os.path.join(base_dir, 'pill_detection*'))

    if not train_folders:
        print("❌ 학습 결과를 찾을 수 없습니다.")
        return result

    # 가장 최근 폴더 선택
    latest_folder = max(train_folders, key=os.path.getctime)
    result['latest_folder'] = latest_folder
    print(f"📂 학습 결과 폴더: {latest_folder}")

    # 모델 경로 설정
    best_model_path = os.path.join(latest_folder, 'weights', 'best.pt')
    last_model_path = os.path.join(latest_folder, 'weights', 'last.pt')

    if os.path.exists(best_model_path):
        result['best_model_path'] = best_model_path
        print(f"✅ 최고 성능 모델: {best_model_path}")
    if os.path.exists(last_model_path):
        result['last_model_path'] = last_model_path
        print(f"✅ 최종 모델: {last_model_path}")

    # 메트릭 읽기
    results_csv = os.path.join(latest_folder, 'results.csv')
    if os.path.exists(results_csv):
        try:
            df = pd.read_csv(results_csv)
            last_row = df.iloc[-1]

            metrics_to_show = {
                'train/box_loss': '훈련 박스 손실',
                'train/cls_loss': '훈련 분류 손실',
                'val/box_loss': '검증 박스 손실',
                'val/cls_loss': '검증 분류 손실',
                'metrics/mAP50(B)': 'mAP50',
                'metrics/mAP50-95(B)': 'mAP50-95'
            }

            print(f"\n📊 학습 메트릭 (마지막 에포크 기준):")
            for key, label in metrics_to_show.items():
                if key in df.columns:
                    value = last_row[key]
                    result['metrics'][label] = round(value, 3)
                    print(f"  {label}: {value:.3f}")

        except Exception as e:
            print(f"⚠️ CSV 파일 읽기 오류: {e}")

    return result

### 추론 실행 및 결과 CSV 출력

In [ ]:
import pandas as pd

def get_unique_filename(base_dir, base_name="result", ext=".csv"):
    """
    지정된 디렉토리에서 중복되지 않는 고유한 파일 이름을 생성하여 반환합니다.

    이 함수는 base_name과 확장자(ext)를 기준으로 result1.csv, result2.csv, ... 형태의
    파일 이름을 순차적으로 생성하며, 이미 존재하지 않는 첫 번째 파일 이름을 반환합니다.

    예:
        base_dir = "./outputs", base_name = "result", ext = ".csv"
        이미 result1.csv, result2.csv가 존재하면 → result3.csv를 반환합니다.

    Args:
        base_dir (str): 파일을 생성할 기준 디렉토리 경로
        base_name (str): 생성할 파일의 기본 이름 (기본값: "result")
        ext (str): 파일 확장자 (기본값: ".csv")

    Returns:
        tuple:
            - full_path (str): 생성된 고유 파일 이름의 전체 경로
            - candidate (str): 생성된 고유 파일 이름 (예: "result3.csv")
    """
    index = 1
    while True:
        candidate = f"{base_name}{index}{ext}"
        full_path = os.path.join(base_dir, candidate)
        if not os.path.exists(full_path):
            return full_path, candidate  # 저장 경로와 파일명
        index += 1

def run_inference_and_save_csv(model_path, test_path, category_id_map, output_path):
    """
    학습된 YOLO 모델을 사용해 테스트 이미지에 대한 객체 탐지를 수행하고,
    결과를 COCO 스타일의 CSV 파일로 저장합니다.

    이 함수는 다음 작업을 수행합니다:
        - 지정된 YOLO 모델(.pt)을 불러와 테스트 이미지 디렉토리에서 객체 탐지를 수행
        - 예측된 결과를 기반으로 COCO 포맷과 유사한 형식의 CSV를 생성
        - category_id는 YOLO 내부 클래스 인덱스를 COCO의 category_id로 다시 매핑하여 저장
        - 이미지 파일명에서 숫자만 추출하여 image_id로 사용
        - 결과는 고유한 파일 이름(`result1.csv`, `result2.csv` 등)으로 저장됨

    Args:
        model_path (str): 학습된 YOLO 모델(.pt) 파일 경로
        test_path (str): 테스트 이미지가 저장된 디렉토리 경로
        category_id_map (dict): COCO의 category_id → YOLO 클래스 인덱스 매핑 딕셔너리
        output_path (str): 추론 결과 CSV 파일을 저장할 디렉토리 경로

    Returns:
        str: 저장된 결과 CSV 파일명 (예: 'result1.csv')

    Notes:
        - CSV 컬럼: ["annotation_id", "image_id", "category_id", "bbox_x", "bbox_y", "bbox_w", "bbox_h", "score"]
        - image_id는 이미지 파일명에서 숫자만 추출하여 생성됨 (예: 'K-123456_...' → 123456)
        - annotation_id는 모든 탐지 박스에 대해 1부터 순차 증가
    """
    model = YOLO(model_path)
    reverse_map = {v: k for k, v in category_id_map.items()}

    results = model.predict(source=test_path, stream=True)
    rows = []
    ann_id = 1

    for r in results:
        file_name = os.path.basename(r.path)
        image_id = int("".join(filter(str.isdigit, file_name)))
        for box in r.boxes:
            cls_idx = int(box.cls.item())
            coco_category_id = reverse_map[cls_idx]
            x, y, w, h = box.xywh[0].tolist()
            score = round(float(box.conf.item()), 2)
            rows.append([ann_id, image_id, coco_category_id, int(x), int(y), int(w), int(h), score])
            ann_id += 1

    df = pd.DataFrame(rows, columns=["annotation_id", "image_id", "category_id", "bbox_x", "bbox_y", "bbox_w", "bbox_h", "score"])

    # ✅ 유니크한 파일 경로 생성
    output_csv_path, filename_only = get_unique_filename(output_path)

    df.to_csv(output_csv_path, index=False)
    print(f"[INFO] 추론 결과 저장 완료: {output_csv_path}")
    return filename_only


## 전체 실행 흐름 요약

In [ ]:
# 1. 병합
image_to_annots, all_categories = merge_annotations_by_image(train_ann_path)

총 JSON 파일 수: 4526


병합 중: 100%|██████████| 4526/4526 [00:05<00:00, 838.02it/s]


In [ ]:
# 2. category_id YOLO 인덱스 매핑
category_id_map = {cid: idx for idx, cid in enumerate(sorted(all_categories.keys()))}

In [ ]:
# 3. 분할
train_keys, val_keys = split_dataset(image_to_annots)

In [ ]:
# 4. 변환 및 복사 : train, val
convert_to_yolo_format(
    {k: image_to_annots[k] for k in train_keys},
    all_categories,
    category_id_map,
    f"{yolo_path}/images/train",
    f"{yolo_path}/labels/train",
    train_img_path
    )

convert_to_yolo_format(
    {k: image_to_annots[k] for k in val_keys},
    all_categories,
    category_id_map,
    f"{yolo_path}/images/val",
    f"{yolo_path}/labels/val",
    train_img_path
)

Converting: 100%|██████████| 298/298 [00:07<00:00, 40.76it/s]


In [ ]:
# 5. dataset.yaml 저장
save_dataset_yaml(yolo_path, category_id_map)

In [ ]:
# 6. 학습 실행 (위 코드에서 진행함)
# 6. 학습 결과 출력 함수 실행
analysis_result = analyze_latest_yolo_training_result()

# 필요한 경우 best.pt 경로 활용
best_pt = analysis_result.get('best_model_path')
if best_pt:
    print(f"\n📌 사용할 모델: {best_pt}")

📂 학습 결과 폴더: result/train/pill_detection3


In [ ]:
# 7. 추론 실행 및 결과 저장
run_inference_and_save_csv(
    # model_path = best_pt
    model_path = f"{project_path}/result/train/pill_detection10/weights/best.pt", # best_pt가 정확하지 않을 때 직접 지정
    test_path=test_img_path,
    category_id_map=category_id_map,
    output_path=project_path          # 저장할 디렉토리 지정
)


image 1/843 /content/drive/MyDrive/Codeit_AI_4th_Drug_image_CV_project/data/test_images/1.png: 1280x992 1 0, 1 23, 1 44, 1 51, 79.3ms
image 2/843 /content/drive/MyDrive/Codeit_AI_4th_Drug_image_CV_project/data/test_images/10.png: 1280x992 1 0, 1 22, 1 37, 1 55, 57.0ms
image 3/843 /content/drive/MyDrive/Codeit_AI_4th_Drug_image_CV_project/data/test_images/100.png: 1280x992 1 0, 1 22, 1 25, 1 63, 57.2ms
image 4/843 /content/drive/MyDrive/Codeit_AI_4th_Drug_image_CV_project/data/test_images/1003.png: 1280x992 1 3, 1 20, 1 49, 1 67, 53.6ms
image 5/843 /content/drive/MyDrive/Codeit_AI_4th_Drug_image_CV_project/data/test_images/1004.png: 1280x992 1 3, 1 20, 1 49, 1 67, 53.0ms
image 6/843 /content/drive/MyDrive/Codeit_AI_4th_Drug_image_CV_project/data/test_images/1005.png: 1280x992 1 3, 1 20, 1 49, 1 67, 55.5ms
image 7/843 /content/drive/MyDrive/Codeit_AI_4th_Drug_image_CV_project/data/test_images/1006.png: 1280x992 1 3, 1 33, 1 45, 1 47, 52.9ms
image 8/843 /content/drive/MyDrive/Codeit_AI_4

'result9.csv'